## Participant-level analysis pipeline for calculating frontal alpha asymmetry in the *MusEEG* project

<span style='color: blue'>Version 1.2 (2023-07-24)<br></span>Created by Dr. Daniel Holt (APKS), modified and extended by Dr. Torsten Wüstenberg (CNSR).

**Aim:** Compute Frontal Alpha Asymmetry (FAA) for electrode sites F3, F4, F7 and F8.

**Gereral analysis steps:**

1) *Specify and load data for analysis*
2) *Specify onsets and durations of relevant periods*
3) *Re-referencing to an avrage reference*
4) *Manual data quality assessment*
5) *Compute FAA and save results*


## Preparations 

### Import libraries

The analysis pipeline is based on the following libraries. In case of an error in the execution of this cell, probably one or more of the libraries is not installed. In this case, start a terminal in **ANACONDA.NAVIGATOR** *Environments>Terminal* and install the library in question using the command </br><span style='color: red'>*pip install [library name]*</span>.

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipyfilechooser

mne.set_log_level("ERROR")  # only show errors (no warnings or information messages)
results_csv = "FAA_Results.csv"  # save analysis results to this file

### Core function for FAA-computation

Use the Filter-Hilbert-method to compute signal strength in the Alpha band (8 to 13 Hz), pooled for the left (F3 & F7) and right electrodes (F4 & F8).

FAA is calculated using the formula:

**FAA = {𝜶(R) - 𝜶(L)} / {𝜶(R) + 𝜶(L)}**

### Channel information and display configuration

The first line of the cell below enables interactive plots displayed in separate windows. Change to `%matplotlib inline` for non-interactive plots inside this notebook.

In [ ]:
%matplotlib qt
ch_type_mapping = {
    "F3": "eeg",
    "F4": "eeg",
    "F7": "eeg",
    "F8": "eeg",
    "vEOG": "eog",
    "hEOG": "eog",
}
scal = {"eeg": 200e-6, "eog": 1000e-6}  # use e-6 for display in microvolts

## Pipeline 

### 1) Select and load data for analysis

Run the cell below and click "Select" to choose the BrainVision **.vhdr** file of the data set to be analyzed. The participant ID defaults to the file name, change the line `participant = ...` for setting a different value. Since the analysis focuses on the relative activity of right and left hemispheres, an average reference is suitable despite the small number of electrodes.

In [ ]:
fc = ipyfilechooser.FileChooser(".")
display(fc)

In [ ]:
participant = fc.selected_filename[:-5]
raw = mne.io.read_raw_brainvision(fc.selected, preload=True)
raw.set_channel_types(ch_type_mapping)
raw.set_eeg_reference("average")
raw.filter(l_freq=1, h_freq=None)

### 2) Specify analysis segments

Positive music was coded as 'Stimulus/S  1', negative music as 'Stimulus/S  2', each condition occurs twice in the data set. Analyse a segment of 30 to 150 secs (i.e., onset 30 and duration 120 secs) for each piece of music. 'Stimulus/S  3' was a resting phase at the beginning.

In [ ]:
# select all event markers ending in 1, 2 or 3
events, _ = mne.events_from_annotations(raw, regexp=".*[123]$")
# code first and second presentation of each condition separately
events[np.where(events[:, 2] == 1)[0][-1], 2] = 10
events[np.where(events[:, 2] == 2)[0][-1], 2] = 20
event_id = {"Ruhe": 3, "Positiv_1": 1, "Positiv_2": 10, "Negativ_1": 2, "Negativ_2": 20}
# shift onset of analysis epochs by 30 secs
events[:, 0] = events[:, 0] + 30000
epochs = mne.Epochs(raw, events, event_id, tmax=120, preload=True)
epochs.plot(
    events=events, scalings=scal, title="1/10 = Positiv, 2/20 = Negativ, 3 = Ruhe"
)

### 3) Reduce ocular artifacts

Due to the robustness of FAA, ocular artifact correction is not required, but it is good standard practice. Use standard regression correction trained on automatically identified EOG events.

In [ ]:
eog_epochs = mne.preprocessing.create_eog_epochs(raw)
eog_evoked = eog_epochs.average("all")
model_evoked = mne.preprocessing.EOGRegression(picks="eeg", picks_artifact="eog").fit(
    eog_evoked
)
epochs_clean = model_evoked.apply(epochs)
epochs_clean.plot(
    events=events, scalings=scal, title="1/10 = Positiv, 2/20 = Negativ, 3 = Ruhe"
)

### 4) Manual data quality assessment

Exclude bad blocks based on the plot generated in the previous step.

In [ ]:
bad_epochs_input = input(
    "Bitte geben Sie die /Epoch Number/ auszuschliessender Blöcke ein (durch Kommata getrennt): "
)

In [ ]:
bad_epochs = [int(epoch_num) for epoch_num in bad_epochs_input.split(",")]
# epochs_clean.drop([int(epoch_num) for epoch_num in bad_epochs.split(',')])
# epochs_clean.plot(events=events, scalings = scal, title='1/10 = Positiv, 2/20 = Negativ, 3 = Ruhe')

### 5) Compute FAA and save results

In [ ]:
alpha = epochs_clean.copy()
alpha.filter(l_freq=8, h_freq=13)
alpha.apply_hilbert(envelope=True)
alpha.plot(scalings=10e-6)
left_alpha = np.array([np.mean(epoch) for epoch in alpha.get_data(["F3", "F7"])])
right_alpha = np.array([np.mean(epoch) for epoch in alpha.get_data(["F4", "F8"])])
FAA = (right_alpha - left_alpha) / (right_alpha + left_alpha)
FAA[bad_epochs] = np.nan

**Save results**

In [ ]:
results = pd.read_csv(results_csv)
faa_analysis = pd.DataFrame(FAA.round(5).reshape((1, 5)), columns=event_id.keys())
faa_analysis.insert(0, "Vpn", participant)
results = pd.concat([results, faa_analysis])
results.to_csv(results_csv, index=False)
print(f'Analyse-Ergebnis in "{results_csv}" gespeichert.')
faa_analysis